In [1]:
import keras
import keras.backend as K
from keras.layers import Input, Dense, AveragePooling2D, Flatten, Lambda
from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
import numpy as np

In [2]:
# Global weight decay value
weight_decay = 1e-4

In [3]:
def Vggface2_ResNet50(input_dim=(224, 224, 3), nb_classes=8631):
    # Input layer
    inputs = Input(shape=input_dim, name='base_input')

    # Backbone (match layer names to weights file)
    from keras.applications import ResNet50
    base_model = ResNet50(weights=None, include_top=False, input_tensor=inputs)
    x = base_model.output

    # Average pooling layer
    x = AveragePooling2D((7, 7), name='avg_pool')(x)
    x = Flatten(name='flatten_1')(x)

    # Fully connected projection layer
    x = Dense(512, activation='relu', name = 'dim_proj')(x)

    # Classifier layer
    y = Dense(
        nb_classes,
        activation='softmax',
        use_bias=False,
        trainable=True,
        kernel_initializer='orthogonal',
        kernel_regularizer=l2(weight_decay),
        name='classifier_low_dim'
    )(x)

    # Create model
    model = Model(inputs=inputs, outputs=y)
    return model

In [4]:
# Path to weights file
weights_path = "/Users/ifeanyinwaneri/Downloads/vggface2_Keras/model/resnet50_softmax_dim512/weights.h5"

# Initialize the model
model = Vggface2_ResNet50(input_dim=(224, 224, 3), nb_classes=8631)

# Save the initial weights of a specific layer for comparison
layer_name = 'dim_proj'
initial_weights = model.get_layer(layer_name).get_weights()  # Save before loading weights

# Load the pretrained weights by layer name
model.load_weights(weights_path, by_name=True)

# Get the weights after loading
loaded_weights = model.get_layer(layer_name).get_weights()

# Verify if weights were loaded successfully
if not all(np.array_equal(iw, lw) for iw, lw in zip(initial_weights, loaded_weights)):
    print(f"Weights successfully loaded for layer '{layer_name}'.")
else:
    print(f"Weights NOT loaded for layer '{layer_name}'.")

2024-12-09 08:54:14.960585: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-12-09 08:54:14.960672: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 128.00 GB
2024-12-09 08:54:14.960686: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 48.00 GB
2024-12-09 08:54:14.960728: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-09 08:54:14.960761: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Weights successfully loaded for layer 'dim_proj'.


In [5]:
# Compile the model
model.compile(
    optimizer=SGD(learning_rate=0.1, momentum=0.9, nesterov=True),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary to confirm successful loading
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ base_input          │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ base_input[0][0]  │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 29,055,872 (110.84 MB)

 Trainable params: 29,002,752 (110.64 MB)

 Non-trainable params: 53,120 (207.50 KB)

# Evaluating the model on VGGFace Dataset 

In [ ]:
import torch


In [ ]:
pretrained model  =  "/Users/ifeanyinwaneri/Documents/Documents - Ifeanyi’s MacBook Pro/Computer-Vision-Model/face-emotion-recognition/models/pretrained_faces/state_vggface2_enet0_new.pt"